## Building Docker Artifacts

In [ ]:
cd farmer

In [ ]:
!docker login -u username -p password

In [ ]:
%%writefile config.py
gs_bucket_name="data-labeling-demo"
Bucket_uri="gs://data-labeling-demo"
version=1
store_artifacts=Bucket_uri + "/" + str(version)
data_path=Bucket_uri + "/" + "data/data_raw.csv"
processed_data=Bucket_uri + "/" + "processed/data_processed.csv"

In [ ]:
%%writefile Dockerfile
FROM tensorflow/tensorflow:2.2.0-gpu
ARG DEBIAN_FRONTEND=noninteractive
# Install apt dependencies
RUN apt-get update && apt-get install -y \
    git \
    gpg-agent \
    python3-cairocffi \
    protobuf-compiler \
    python3-pil \
    python3-lxml \
    python3-tk \
    wget
# Install gcloud and gsutil commands
# https://cloud.google.com/sdk/docs/quickstart-debian-ubuntu
RUN export CLOUD_SDK_REPO="cloud-sdk-$(lsb_release -c -s)" && \
    echo "deb http://packages.cloud.google.com/apt $CLOUD_SDK_REPO main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && \
    curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - && \
    apt-get update -y && apt-get install google-cloud-sdk -y
WORKDIR /pipeline
COPY ./ ./
RUN pip install -r requirements.txt
RUN pip install "dask[dataframe]" --upgrade
ENV TF_CPP_MIN_LOG_LEVEL 3

## Build Docker Image

In [ ]:
!docker build -t ankitaj86/kubeflow-sdk-farmer .

## Push Docker Image to Docker hub

In [ ]:
!docker push ankitaj86/kubeflow-sdk-farmer

## Install Kubeflow SDk

In [ ]:
!pip install kfp

# Building Kubeflow Pipeline

In [ ]:
import kfp
import kfp.components as comp
from kubernetes.client.models import V1EnvVar

## Define kubeflow pipeline Component

In [ ]:
@kfp.dsl.component
def get_data():
    # Defining component configuration
    getdata_component = kfp.dsl.ContainerOp(
        name='Data-Preparation',
        image='docker.io/ankitaj86/kubeflow-sdk-farmer',
        command=['python', 'get_data.py'],
        )
    return getdata_component

In [ ]:
@kfp.dsl.component
def data_preprocessing():    
    # Defining component configuration
    data_preprocessing = kfp.dsl.ContainerOp(
        name='data-preprocessing',
        image='docker.io/ankitaj86/kubeflow-sdk-farmer',
        command=['python', 'process_data.py'],
        )
    return data_preprocessing

In [ ]:
@kfp.dsl.component
def training():
    # Defining component configuration
    training_component = kfp.dsl.ContainerOp(
        name='training',
        image='docker.io/ankitaj86/kubeflow-sdk-farmer',
        command=['python', 'train.py'],
        file_outputs={'mlpipeline-ui-metadata':'/mlpipeline-ui-metadata.json', "mlpipeline-metrics":'/mlpipeline-metrics.json'}
        )
    
    return training_component

In [ ]:
# Let see output of component configuration
debug = True
if debug :
    training_component_vis = training()
    print(training_component_vis)

## Final Kubeflow pipeline Definition

In [ ]:
@kfp.dsl.pipeline(
  name="Modeling Swiss farmer's attitudes about",
  description="Modeling Swiss farmer's attitudes about"
)
def farmer():
    download_data = get_data()
    download_data.execution_options.caching_strategy.max_cache_staleness = "P0D"
    data_processing = data_preprocessing().after(download_data)
    data_processing.execution_options.caching_strategy.max_cache_staleness = "P0D"
    train = training().after(data_processing)
    train.execution_options.caching_strategy.max_cache_staleness = "P0D"


In [ ]:
# Let see output of pipeline configuration
debug = True
if debug :
    training_pipeline_output = farmer()
    print(training_pipeline_output)

## Compile Kubeflow Pipeline 
* It will Generate .zip file inside this contain YAMl file which contain the configuration of kubeflow pipeline

In [ ]:
kfp.compiler.Compiler().compile(farmer, 'farmer-kubeflow-sdk-demo.zip')

## Connect to deployed kubeflow pipeline Endpoint (GCP)

In [ ]:
# Create kfp client
# Note: Add the KubeFlow Pipeline endpoint below if the client is not running on the same cluster.
client = kfp.Client("57f227181c3ac171-dot-us-central1.pipelines.googleusercontent.com")

### Create Experiment 

In [ ]:
EXPERIMENT_NAME = 'farmer-1'
experiment = client.create_experiment(name=EXPERIMENT_NAME)

### Deploy pipeline to kubeflow pipeline Endpoint

In [ ]:
run = client.run_pipeline(experiment.id, 'farmer-run-3', 'farmer-kubeflow-sdk-demo.zip')